# Downloading JAXA Precipication Dataset (daily)

Laurenz Schindler

## Dataset Information [1]

GSMaP Hourly dataset is obtained from the Dual-frequency Precipitation Radar (DPR) sensor onboard Global Precipitation Measurement (GPM), other GPM constellation satellites, and Geostationary satellites produced by the Japan Aerospace Exploration Agency (JAXA).
The GSMaP is generated based on a multi-satellite algorithm under the GPM mission, and the accuracy has been improved by DPR data and information. It offers a map of global precipitation by combining: estimated precipitation based on multiple microwave radiometers (imager/sounder) and cloud moving information obtained from geostationary infrared (IR) data.
The GSMaP algorithm can be roughly divided into the following three algorithms: microwave imager (MWI) algorithm, microwave sounder (MWS) algorithm, and microwave-Infrared (IR) combined (MVK) algorithm. A global satellite mapping of precipitation can be subject to standard processing or near real-time processing.
In standard processing, hourly observation data is processed then the data is averaged monthly. Near real-time processing provides a higher data frequency than standard processing (every hour). The provided formats are HDF5, text, GeoTIFF and NetCDF. The Sampling resolution is 0.1 degree grid. The projection method is EQR.
The statistical period is 1 hourly. The current version of the product is Version 5. The Version 4 is also available. The generation unit is global.

[1] Japan Aerospace Exploration Agency. 1998. GSMaP(Hourly). https://doi.org/10.57746/EO.01gs73bkt358gfpy92y2qns5e9


## Notebook Overview

This notebook is a guide on retrieving the precipitation dataset (daily interval) from the JAXA Earth API. For example purposes, the data for May 2024 will be loaded. 
...

## Installing JAXA Api

Following the [instructions](https://data.earth.jaxa.jp/api/python/v0.1.4/en/quick.html), download the package .zip file and install it using pip:

In [4]:
!pip install ../../jaxa-earth-0.1.4.zip

Processing d:\documents\uzk_master\earth_sys_data\jaxa-earth-0.1.4.zip
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for jaxa-earth: filename=jaxa_earth-0.1.4-py3-none-any.whl size=47470 sha256=f2ec1583e13dfe399cee1d5ce2a92f9ea42ace15083d8aaf5bb5c95e56af83f0
  Stored in directory: c:\users\ameri\appdata\local\pip\cache\wheels\a2\52\3a\965cce88c8acd75f38dfcf5a70161d970d4d6a7572866ce7ec
Successfully built jaxa-earth


In [5]:
from jaxa.earth import je

## Retrieving the data


### Structure of Data in API

First, let us look at the complete list of data-collections and bands for each collection:

In [6]:
collections, bands = je.ImageCollectionList(ssl_verify=True).filter_name()

 - Geting image collection information : completed
 - Searching : 96 image collections found!


The obtained `collections` and `bands` are two lists of the same length. `collections` contains the 
names for all data-collections, while at the position in the list, in `bands` a list of available bands is given for this collection.

From the dataset-list on the website, we know the exact name of this dataset. To be able to load it using the API, we also need to identify the bands available in this dataset, which is a required parameter for the request-call.

In [7]:
precip_collection = "JAXA.EORC_GSMaP_standard.Gauge.00Z-23Z.v6_daily"
precip_index = collections.index(precip_collection)

bands[precip_index]

['PRECIP']

### Loading data into memory

There are multiple parameters to specify exactly what data is to be obtained:

- **`collection`**: The unique identifier for the dataset. Can be found in the dataset-list on the website.

- **`dlim`**: Date limit - defines the temporal range as a list of two ISO 8601 formatted datetime strings: `[start_date, end_date]`. Format: `"YYYY-MM-DDTHH:MM:SS"` (note american date format)

- **`ppu`** (pixels per unit): Controls the spatial resolution of the retrieved data, specified as pixels per degree. 

- **`bbox`**: Bounding box defining the spatial extent as `[min_longitude, min_latitude, max_longitude, max_latitude]`. Coordinates must be in decimal degrees using, within (90.0° N, -180.0° W, -90.0° S, 180.0° E)

- **`band`**: The specific variable/band to retrieve from the collection. Each collection may contain multiple bands (variables), only a single can be passed here.

In [55]:
# Load module
from jaxa.earth import je

# Set query parameters (for intial testing)
dlim = ["2024-05-01T00:00:00","2024-06-01T00:00:00"]  # YYYY-MM-DD
ppu  = 5
bbox = [110, 20, 160, 50]

# Get an image
data = je.ImageCollection(collection=precip_collection,ssl_verify=True)\
        .filter_date(dlim=dlim)\
        .filter_resolution(ppu=ppu)\
        .filter_bounds(bbox=bbox)\
        .select(band=bands[precip_index][0])\
        .get_images()

 - Collection : JAXA.EORC_GSMaP_standard.Gauge.00Z-23Z.v6_daily
 - Date : 2024-05/01/, 2024-05/02/, 2024-05/03/, 2024-05/04/, 2024-05/05/, 2024-05/06/, 2024-05/07/, 2024-05/08/, 2024-05/09/, 2024-05/10/, 2024-05/11/, 2024-05/12/, 2024-05/13/, 2024-05/14/, 2024-05/15/, 2024-05/16/, 2024-05/17/, 2024-05/18/, 2024-05/19/, 2024-05/20/, 2024-05/21/, 2024-05/22/, 2024-05/23/, 2024-05/24/, 2024-05/25/, 2024-05/26/, 2024-05/27/, 2024-05/28/, 2024-05/29/, 2024-05/30/, 2024-05/31/, 2024-06/01/, 
 - Resolution : 5.0 pixels per 1 degree 
 - Bounds : [110, 20, 160, 50]
 - Band : PRECIP
 - Loading images No.0 : 2024-05/01/
   ------10------20------30------40------50------60------70------80------90-----100
   ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
 - Loading images No.1 : 2024-05/02/
   ------10------20------30------40------50------60------70------80------90-----100
   ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
 - Loading

In [56]:
images = je.ImageProcess(data)
images.raster.img.shape

(32, 150, 250, 1)

Looking at the shape of the obtained data, we can notice multiple things:

- 32 time-steps, which means the end-date is also inclusive; to obtain only the data for may, the end-time must also still be in May.
- pixel resolution: the given bounding box defines the area as such: `[min_lon, min_lat, max_lon, max_lat]`, thus we have `160 - 110 = 50 degrees` longitude and `50 - 20 = 30 degrees` latitude. Multiplying by our specified `ppu`, we get `30 * 5 = 150` and `50 * 5 = 250`, which is exactly the shape of the array.

Thus we know, if the source dataset has a resolution of 0.1 deg, we would need to set `ppu = 10` to retrieve the data in its original spatial resolution.

In [ ]:
# Process and show an image
img = je.ImageProcess(data)\
        .show_images()

Of course, this pipeline might be nice for small-scale and instantaneous data-analysis, but it is completely unsuitable for any large-scale (machine-learning) applications. To make do, we need to save the data after obtaining from the API.

## Writing to disk and chunking

Thus, the below functions wrap the API call(s) to allow for customizable and easy writing to disk. Since we want to save the data in a suitable file-format, I chose GeoTIFF, utilizing the rasterio library.

In [ ]:
# !pip install rasterio

### Utility function for chunking

In [11]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

def prepare_date_chunks(date_range, chunking="monthly"):

    if chunking not in ("daily", "monthly", "yearly"):
        raise ValueError(f"chunking must be 'daily', 'monthly', or 'yearly', got '{chunking}'")
    
    # Parse input dates
    start_dt = datetime.fromisoformat(date_range[0].replace('Z', '+00:00'))
    end_dt = datetime.fromisoformat(date_range[1].replace('Z', '+00:00'))
    
    # chunking stategies
    strategies = {
        "daily": {
            "next_chunk": lambda dt: dt + timedelta(days=1),
            "name_format": "%Y-%m-%d"
        },
        "monthly": {
            "next_chunk": lambda dt: dt.replace(day=1) + relativedelta(months=1),
            "name_format": "%Y-%m"
        },
        "yearly": {
            "next_chunk": lambda dt: dt.replace(month=1, day=1) + relativedelta(years=1),
            "name_format": "%Y"
        }
    }
    
    strategy = strategies[chunking]
    next_chunk = strategy["next_chunk"]
    name_format = strategy["name_format"]
    
    chunks = []
    current_dt = start_dt
    
    while current_dt < end_dt:
        chunk_start = current_dt
        chunk_end = min(next_chunk(current_dt), end_dt)
        
        name = chunk_start.strftime(name_format)
        dlim = [
            chunk_start.isoformat(),
            (chunk_end - timedelta(seconds=1)).isoformat() # subtract 1 second to include the last moment of the chunk to not include end-date
        ]
        
        chunks.append((name, dlim))
        current_dt = chunk_end
    
    return chunks

## Downloading Function

The below function loads the data from the JAXA Earth API and saves it to disk as GeoTIFF format. Besides simply passing the required parameters to the API-call, there are two more features:

1. Data from multiple bands can be downloaded in the same function call. If no band is give, all possible bands for this dataset will be downloaded. 

2. If not all of the required can be loaded into memory at once, the function will split the time-range into managable chunks and save each chunk independently. The possible options for chunking are 'daily', 'monthly' and 'yearly'.

The directory structure will be as follows:
 `dataset-name/band/chunked-files`

In [58]:
import os
import numpy as np
from jaxa.earth import je
import rasterio
from rasterio.transform import from_bounds
import time

def save_data_to_disk(collection, band, date_range, out_path, chunking="monthly", ppu=2, bbox=[-180, -90, 180, 90]):

    s = time.time()
    try:
        if type(band) is str:
            band = [band]
        elif not band: # load all bands for collection
            collections, bands = je.ImageCollectionList(ssl_verify=True).filter_name()
            coll_index = collections.index(collection)
            band = bands[coll_index]

        date_chunks = prepare_date_chunks(date_range, chunking)

        for b in band:
            for name, dlim in date_chunks:
                
                try:
                    inner_path = os.path.join(out_path, collection.replace('.','_'), b)
                    os.makedirs(inner_path, exist_ok=True)

                    filename = f"{name}.tif"

                    # 1. load data to memory
                    data = je.ImageCollection(collection=collection,ssl_verify=True)\
                                                .filter_date(dlim=dlim)\
                                                .filter_resolution(ppu=ppu)\
                                                .filter_bounds(bbox=bbox)\
                                                .select(band=b)\
                                                .get_images()
                    
                    raster = data.raster  # raster is a Raster object per API doc
                    arr = raster.img  # numpy array (t, h, w, 1)
                    arr = raster.img.squeeze(axis=-1)  # remove last dim

                    # 2. bring to correct format for rasterio
                    min_lon, max_lon = raster.lonlim[0]
                    min_lat, max_lat = raster.latlim[0]
                    num_timesteps, height, width = arr.shape
                    transform = from_bounds(min_lon, min_lat, max_lon, max_lat, width, height)

                    dtype = arr.dtype
                    epsg = data.proj_params.epsg
                    crs = f"EPSG:{epsg}"  # coordinate reference system used

                    # 3. Write to GeoTIFF
                    filename = f"{name}.tif"
                    filepath = os.path.join(inner_path, filename)
                    with rasterio.open(
                        filepath,
                        "w",
                        driver="GTiff",
                        height=height,
                        width=width,
                        count=num_timesteps,  # Each timestep becomes a band
                        dtype=dtype,
                        crs=crs,
                        transform=transform
                    ) as dst:
                        for t in range(num_timesteps):
                            dst.write(arr[t], t + 1)
                
                except Exception as e:
                    print(f"Failed to download data for time-range {dlim} and band {b}: {e}")

    
    except Exception as e:
        print(f"Error: {e}")
        return
    
    finally:
        e = time.time()
        print(f"Execution time: {e - s} seconds")

In [60]:
collection = "JAXA.EORC_GSMaP_standard.Gauge.00Z-23Z.v6_daily"
band = "PRECIP"
date_range = ["2024-05-01T00:00:00","2024-05-31T23:59:00"]
out_path = "../data"
ppu = 5
chunking = "monthly"  # choose from "daily", "monthly", "yearly"
bbox = [-180, -90, 180, 90]

save_data_to_disk(collection, band, date_range, out_path, chunking=chunking, ppu=ppu, bbox=bbox)

 - Collection : JAXA.EORC_GSMaP_standard.Gauge.00Z-23Z.v6_daily
 - Date : 2024-05/01/, 2024-05/02/, 2024-05/03/, 2024-05/04/, 2024-05/05/, 2024-05/06/, 2024-05/07/, 2024-05/08/, 2024-05/09/, 2024-05/10/, 2024-05/11/, 2024-05/12/, 2024-05/13/, 2024-05/14/, 2024-05/15/, 2024-05/16/, 2024-05/17/, 2024-05/18/, 2024-05/19/, 2024-05/20/, 2024-05/21/, 2024-05/22/, 2024-05/23/, 2024-05/24/, 2024-05/25/, 2024-05/26/, 2024-05/27/, 2024-05/28/, 2024-05/29/, 2024-05/30/, 2024-05/31/, 
 - Resolution : 5.0 pixels per 1 degree 
 - Bounds : [-180, -90, 180, 90]
 - Band : PRECIP
 - Loading images No.0 : 2024-05/01/
   ------10------20------30------40------50------60------70------80------90-----100
   ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
 - Loading images No.1 : 2024-05/02/
   ------10------20------30------40------50------60------70------80------90-----100
   ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
 - Loading images No.

Loading and saving the global precipitation data from May 2024 took 269 seconds.

### Load the data from the files

In [61]:
def load_data_from_disk(filepath):
    
    # Read the GeoTIFF
    with rasterio.open(filepath) as src:
        # Read all bands (timesteps) into a numpy array
        arr = src.read()  # Shape: (num_timesteps, height, width)
        
        # Get metadata
        metadata = {
            'transform': src.transform,
            'crs': src.crs,
            'bounds': src.bounds,
            'width': src.width,
            'height': src.height,
            'count': src.count  # number of timesteps
        }
    
    return arr, metadata

In [62]:
filepath = r"..\data\JAXA_EORC_GSMaP_standard_Gauge_00Z-23Z_v6_daily\PRECIP\2024-05.tif"
array, metadata = load_data_from_disk(filepath)

In [63]:
array.shape

(31, 900, 1800)

In [66]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML

fig, ax = plt.subplots(figsize=(12, 6))

# Set color limits
vmin, vmax = np.nanmin(array), np.nanmax(array)

# Initialize with first frame
im = ax.imshow(array[0], cmap='turbo', vmin=vmin, vmax=vmax, aspect='auto')
title = ax.set_title(f'Timestep 0/{array.shape[0]-1}')
plt.colorbar(im, ax=ax)

# Update function
def update(frame):
    im.set_array(array[frame])
    title.set_text(f'Timestep {frame}/{array.shape[0]-1}')
    return [im, title]

# Create and save animation
anim = FuncAnimation(fig, update, frames=array.shape[0], blit=True, interval=200)
plt.close()

HTML(anim.to_jshtml())